<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#DICOM-Images" data-toc-modified-id="DICOM-Images-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>DICOM Images</a></span></li><li><span><a href="#Nifti-Maker" data-toc-modified-id="Nifti-Maker-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Nifti Maker</a></span></li><li><span><a href="#xarray-generation" data-toc-modified-id="xarray-generation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>xarray generation</a></span></li><li><span><a href="#xarray-viewer" data-toc-modified-id="xarray-viewer-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>xarray viewer</a></span></li><li><span><a href="#Model-Training" data-toc-modified-id="Model-Training-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Model Training</a></span></li><li><span><a href="#Slurm-Analysis" data-toc-modified-id="Slurm-Analysis-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Slurm Analysis</a></span></li></ul></div>

# New Data Exploration

## DICOM Images

In [ ]:
%matplotlib inline

In [ ]:
from pathlib import Path
from datetime import datetime
from collections import OrderedDict
import numpy as np
import pickle as pkl
import pandas as pd
import xarray as xr
import SimpleITK as sitk

import holoviews as hv
from holoviews import opts
import panel as pn
import hvplot.pandas
hv.extension('bokeh')
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10,8)

from mre.plotting import patient_series_viewer, chaos_viewer, xr_viewer, hv_dl_vis_chaos
from mre.preprocessing import make_nifti_atlas_v2, make_xr_dataset_for_chaos
from mre.segmentation import ChaosDataset
from mre.train_seg_model import train_seg_model 
from mre import pytorch_arch

from torch.utils.data import Dataset, DataLoader
import torchvision.utils
from torchsummary import summary
import torch
import torch.nn as nn
from collections import defaultdict
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models

%load_ext autoreload
%autoreload 2

In [ ]:
data_dir = Path('/pghbio/dbmi/batmanlab/bpollack/predictElasticity/data/CHAOS/Train_Sets/MR/')

In [ ]:
# patient_series_viewer(data_dir, 'DICOMA/PA1/ST0')
# patient_series_viewer(data_dir, '1', img_type='DICOM_CHAOS')

## Nifti Maker

In [ ]:
#make_nifti_atlas_v2()

In [ ]:
#patient_series_viewer(data_dir, 'NIFTI/01', img_type='NIFTI')

In [ ]:
#chaos_viewer(data_dir, 'NIFTI/03')


## xarray generation

In [ ]:
ls ../data/CHAOS/Train_Sets/MR/NIFTI/

In [ ]:
# patients = ["01",  "03",  "08",  "13",  "19",  "21",  "31",  "33",  "36",  "38",
# "02",  "05",  "10",  "15",  "20",  "22",  "32",  "34",  "37",  "39"] 
# ds = make_xr_dataset_for_chaos(patients, 256, 256, 32, 'chaos')


## xarray viewer

In [ ]:
ds_path = Path(data_dir, 'xarray_chaos.nc')
ds = xr.open_dataset(ds_path)

In [ ]:
xr_viewer(ds, overlay_data='mask')
#ds

## Model Training

In [4]:
# Setup paths
out_dir = '/pghbio/dbmi/batmanlab/bpollack/predictElasticity/data/CHAOS/'

In [5]:
torch.cuda.empty_cache()

subj = '01'
version = None
n_layers = 5
model_cap = 16
channel_growth = True
seq_mode = 'random'
model_arch='3D'
now = datetime.today().strftime('%Y-%m-%d_%H-%M-%S')
if version is None: version = now
#model_version=f'chaos_notebook_test_{version}'
model_version=version
print(now)
inputs, targets, names, model = train_seg_model(data_dir, 'xarray_chaos.nc', out_dir, model_version=model_version, subj=subj, loss='dice', dry_run=False,
                                                transform=True, def_seq_mode=seq_mode, coord_conv=False, step_size=80, num_epochs=200, lr=5e-2, 
                                                model_arch=model_arch, resize=False, n_layers=n_layers, channel_growth=channel_growth,
                                                model_cap=model_cap, batch_size=100, test_seq_mode='all', test_aug=False, train_aug=False, val_aug=False,
                                                bc_weight=0.2)

# model_path = Path(out_dir, 'trained_models', subj, f'model_{model_version}.pkl')
# model = pytorch_arch.GeneralUNet3D(n_layers, 1, model_cap, 1, channel_growth, False, False)
# model_dict = torch.load(model_path, map_location='cpu')
# model_dict = OrderedDict([(key[7:], val) for key, val in model_dict.items()])
# model.load_state_dict(model_dict, strict=True)
# model.eval()

2019-10-15_12-08-14
{'train_trans': True, 'train_clip': True, 'train_aug': False, 'train_sample': 'shuffle', 'val_trans': True, 'val_clip': True, 'val_aug': False, 'val_sample': 'shuffle', 'test_trans': True, 'test_clip': True, 'test_aug': False, 'train_seq_mode': None, 'val_seq_mode': None, 'test_seq_mode': 'all', 'def_seq_mode': 'random', 'seed': 100, 'subj': '01', 'batch_size': 100, 'model_cap': 16, 'lr': 0.05, 'step_size': 80, 'gamma': 0.1, 'num_epochs': 200, 'dry_run': False, 'coord_conv': False, 'loss': 'dice', 'model_arch': '3D', 'n_layers': 5, 'in_channels': 1, 'out_channels_final': 1, 'channel_growth': True, 'transfer_layer': False, 'bce_weight': 0.5, 'resize': False, 'transform': True, 'bc_weight': 0.2}
<xarray.Dataset>
Dimensions:   (sequence: 3, subject: 20, x: 256, y: 256, z: 32)
Coordinates:
  * subject   (subject) object '01' '03' '08' '13' '19' ... '32' '34' '37' '39'
  * sequence  (sequence) object 't1_in' 't1_out' 't2'
  * x         (x) int64 0 1 2 3 4 5 6 7 8 ... 248

train: bce: 0.052528, dice: 0.514802, loss: 0.283665
val: bce: 0.373618, dice: 0.978592, loss: 0.676105
0m 4s
Epoch 44/199
----------
LR 0.05
train: bce: 0.047692, dice: 0.496635, loss: 0.272164
val: bce: 0.385325, dice: 0.982306, loss: 0.683815
0m 4s
Epoch 45/199
----------
LR 0.05
train: bce: 0.046037, dice: 0.483521, loss: 0.264779
val: bce: 0.390438, dice: 0.993430, loss: 0.691934
0m 4s
Epoch 46/199
----------
LR 0.05
train: bce: 0.047057, dice: 0.473515, loss: 0.260286
val: bce: 0.383781, dice: 0.998165, loss: 0.690973
0m 4s
Epoch 47/199
----------
LR 0.05
train: bce: 0.048439, dice: 0.453251, loss: 0.250845
val: bce: 0.294697, dice: 0.914505, loss: 0.604601
0m 4s
Epoch 48/199
----------
LR 0.05
train: bce: 0.045627, dice: 0.449227, loss: 0.247427
val: bce: 0.168671, dice: 0.652725, loss: 0.410698
saving best model
0m 4s
Epoch 49/199
----------
LR 0.05
train: bce: 0.043766, dice: 0.425392, loss: 0.234579
val: bce: 0.157618, dice: 0.698276, loss: 0.427947
0m 4s
Epoch 50/199
-------

train: bce: 0.027791, dice: 0.213112, loss: 0.120451
val: bce: 0.064052, dice: 0.279378, loss: 0.171715
saving best model
0m 4s
Epoch 98/199
----------
LR 0.005000000000000001
train: bce: 0.027270, dice: 0.211068, loss: 0.119169
val: bce: 0.064120, dice: 0.278866, loss: 0.171493
saving best model
0m 4s
Epoch 99/199
----------
LR 0.005000000000000001
train: bce: 0.027446, dice: 0.212504, loss: 0.119975
val: bce: 0.065366, dice: 0.278105, loss: 0.171735
0m 4s
Epoch 100/199
----------
LR 0.005000000000000001
train: bce: 0.027481, dice: 0.208258, loss: 0.117870
val: bce: 0.066832, dice: 0.277705, loss: 0.172268
0m 4s
Epoch 101/199
----------
LR 0.005000000000000001
train: bce: 0.027020, dice: 0.209347, loss: 0.118183
val: bce: 0.069159, dice: 0.278378, loss: 0.173768
0m 4s
Epoch 102/199
----------
LR 0.005000000000000001
train: bce: 0.029206, dice: 0.212551, loss: 0.120879
val: bce: 0.069809, dice: 0.278650, loss: 0.174229
0m 4s
Epoch 103/199
----------
LR 0.005000000000000001
train: bce: 

train: bce: 0.024482, dice: 0.186301, loss: 0.105392
val: bce: 0.058758, dice: 0.267012, loss: 0.162885
0m 4s
Epoch 148/199
----------
LR 0.005000000000000001
train: bce: 0.024463, dice: 0.184288, loss: 0.104376
val: bce: 0.059837, dice: 0.264364, loss: 0.162101
0m 4s
Epoch 149/199
----------
LR 0.005000000000000001
train: bce: 0.024918, dice: 0.188123, loss: 0.106521
val: bce: 0.060774, dice: 0.264177, loss: 0.162476
0m 4s
Epoch 150/199
----------
LR 0.005000000000000001
train: bce: 0.025383, dice: 0.183344, loss: 0.104364
val: bce: 0.060312, dice: 0.266071, loss: 0.163191
0m 4s
Epoch 151/199
----------
LR 0.005000000000000001
train: bce: 0.025375, dice: 0.184864, loss: 0.105120
val: bce: 0.059346, dice: 0.268103, loss: 0.163725
0m 4s
Epoch 152/199
----------
LR 0.005000000000000001
train: bce: 0.024017, dice: 0.183825, loss: 0.103921
val: bce: 0.058801, dice: 0.268544, loss: 0.163673
0m 4s
Epoch 153/199
----------
LR 0.005000000000000001
train: bce: 0.023541, dice: 0.183441, loss: 0.

train: bce: 0.023578, dice: 0.176892, loss: 0.100235
val: bce: 0.059997, dice: 0.263386, loss: 0.161691
0m 4s
Epoch 199/199
----------
LR 0.0005000000000000001
train: bce: 0.023240, dice: 0.174557, loss: 0.098899
val: bce: 0.060422, dice: 0.263270, loss: 0.161846
0m 4s
Best val loss: 0.160755


In [6]:
model_pred = None
if model:
    # inputs.to('cuda:0')
    model_pred = torch.zeros_like(inputs)
    for i in range(inputs.shape[0]):
        for j in range(inputs.shape[1]):
            model_pred[i, j, :] = model(inputs[i:i+1, j:j+1, :])
            model_pred[i, j, :] = F.sigmoid(model_pred[i, j, :])
            ones = torch.ones_like(model_pred[i, j, :])
            zeros = torch.zeros_like(model_pred[i, j, :])
        # model_pred[:, i, :] = torch.where(model_pred[:, i, :]>3e-3, ones, zeros)
    inputs.to('cpu')
hv_dl_vis_chaos(inputs, targets, names, ['t1_in', 't1_out', 't2'], model_pred)
# hv_dl_vis_chaos(inputs, targets, names, ['seq'], model_pred)

/home/bpollack/conda_envs/new_mre/lib/python3.6/site-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


:Layout
   .DynamicMap.I   :DynamicMap   [subject,sequence,z]
      :Image   [x,y]   (inputs)
   .DynamicMap.II  :DynamicMap   [subject,sequence,z]
      :Image   [x,y]   (targets)
   .DynamicMap.III :DynamicMap   [subject,sequence,z]
      :Image   [x,y]   (predictions)

## Slurm Analysis

In [ ]:
config_path = Path(out_dir, 'config')

In [ ]:
df = pd.DataFrame()
for f in list(config_path.glob('*2019-10-03_12-10-03*.pkl')):
    s_tmp = pd.Series(pd.read_pickle(str(f)), name=f.stem)
    df = df.append(s_tmp, ignore_index=False)

In [ ]:
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
df['test_dice_mean'] = (df.test_dice_t1_in+df.test_dice_t1_out+df.test_dice_t2)/3.0

In [ ]:
df1 = df.query('channel_growth==1').sort_values('test_dice_t1_out').reset_index().rename(columns={'index':'job_name'})
df2 = df.query('channel_growth==0').sort_values('test_dice_t1_out').reset_index().rename(columns={'index':'job_name'})
(df1.hvplot.line(x='index', y='test_dice_t1_out', hover_cols=['model_cap', 'def_seq_mode', 'bce_weight'], label='C Growth')*
df2.hvplot.line(x='index', y='test_dice_t1_out', hover_cols=['model_cap', 'def_seq_mode', 'bce_weight'], label='C Static')).opts(legend_position='top_left', show_legend=True)

In [ ]:
df1 = df.query('def_seq_mode=="t1_in"').sort_values('test_dice_t1_out').reset_index().rename(columns={'index':'job_name'})
df2 = df.query('def_seq_mode=="t1_out"').sort_values('test_dice_t1_out').reset_index().rename(columns={'index':'job_name'})
df3 = df.query('def_seq_mode=="t2"').sort_values('test_dice_t1_out').reset_index().rename(columns={'index':'job_name'})
df4 = df.query('def_seq_mode=="random"').sort_values('test_dice_t1_out').reset_index().rename(columns={'index':'job_name'})
(df1.hvplot.line(x='index', y='test_dice_t1_out', hover_cols=['model_cap', 'def_seq_mode', 'bce_weight', 'job_name'], label='t1_in')*
df2.hvplot.line(x='index', y='test_dice_t1_out', hover_cols=['model_cap', 'def_seq_mode', 'bce_weight', 'job_name'], label='t1_out')*
df3.hvplot.line(x='index', y='test_dice_t1_out', hover_cols=['model_cap', 'def_seq_mode', 'bce_weight', 'job_name'], label='t2')*
df4.hvplot.line(x='index', y='test_dice_t1_out', hover_cols=['model_cap', 'def_seq_mode', 'bce_weight', 'job_name'], label='random')
).opts(legend_position='top_left', show_legend=True)

In [ ]:
df1 = df.query('bce_weight==0.2').sort_values('best_loss').reset_index().rename(columns={'index':'job_name'})
df3 = df.query('bce_weight==0.5').sort_values('best_loss').reset_index().rename(columns={'index':'job_name'})
df4 = df.query('bce_weight==0.8').sort_values('best_loss').reset_index().rename(columns={'index':'job_name'})
(df1.hvplot.line(x='index', y='best_loss', hover_cols=['model_cap', 'def_seq_mode', 'bce_weight'], label='0.2')*
df3.hvplot.line(x='index', y='best_loss', hover_cols=['model_cap', 'def_seq_mode', 'bce_weight'], label='0.5')*
df4.hvplot.line(x='index', y='best_loss', hover_cols=['model_cap', 'def_seq_mode', 'bce_weight'], label='0.8')
).opts(legend_position='top_left', show_legend=True)

In [ ]:
#
df0 = df.query('model_cap==4').sort_values('test_dice_mean').reset_index().rename(columns={'index':'job_name'})
df1 = df.query('model_cap==8').sort_values('test_dice_mean').reset_index().rename(columns={'index':'job_name'})
df2 = df.query('model_cap==12').sort_values('test_dice_mean').reset_index().rename(columns={'index':'job_name'})
df3 = df.query('model_cap==16').sort_values('test_dice_mean').reset_index().rename(columns={'index':'job_name'})
df4 = df.query('model_cap==32').sort_values('test_dice_mean').reset_index().rename(columns={'index':'job_name'})
(df0.hvplot.line(x='index', y='test_dice_mean', hover_cols=['model_cap', 'def_seq_mode', 'bce_weight'], label='4')*
df1.hvplot.line(x='index', y='test_dice_mean', hover_cols=['model_cap', 'def_seq_mode', 'bce_weight'], label='8')*
df2.hvplot.line(x='index', y='test_dice_mean', hover_cols=['model_cap', 'def_seq_mode', 'bce_weight'], label='12')*
df3.hvplot.line(x='index', y='test_dice_mean', hover_cols=['model_cap', 'def_seq_mode', 'bce_weight'], label='16')*
df4.hvplot.line(x='index', y='test_dice_mean', hover_cols=['model_cap', 'def_seq_mode', 'bce_weight'], label='32')
).opts(legend_position='top_left', show_legend=True)

In [ ]:
df.groupby(['model_cap', 'def_seq_mode'])['test_dice_t1_out'].mean()

Notes: t1_out seems to outperform all other combos (including random).  Best current overall: t1_out, model_cap=8.  Why would adding additional images decrease performance?

In [ ]:
#df.query('model_cap==8 and def_seq_mode=="t1_out" and subj=="01"')

In [ ]:
df1 = df.query('n_layers==5').sort_values('test_dice_t1_out').reset_index().rename(columns={'index':'job_name'})
df2 = df.query('n_layers==6').sort_values('test_dice_t1_out').reset_index().rename(columns={'index':'job_name'})
df3 = df.query('n_layers==7').sort_values('test_dice_t1_out').reset_index().rename(columns={'index':'job_name'})
(df1.hvplot.line(x='index', y='test_dice_t1_out', hover_cols=['model_cap', 'def_seq_mode', 'bce_weight'], label='5')*
df2.hvplot.line(x='index', y='test_dice_t1_out', hover_cols=['model_cap', 'def_seq_mode', 'bce_weight'], label='6')*
df3.hvplot.line(x='index', y='test_dice_t1_out', hover_cols=['model_cap', 'def_seq_mode', 'bce_weight'], label='7')
).opts(legend_position='top_left', show_legend=True)